In [ ]:
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16
# IPython Libraries
import IPython
import ipywidgets
from ipywidgets import interact, interactive, interact_manual

In [ ]:
def plot_data(X, Y, fig=None, options=dict()):
    # fig_data = plt.figure()
    if fig is None:
        fig = plt.subplot(111)
    fig.plot(X, Y, options.get('marker', 'b*'), 
        label=options.get('label', 'Raw data'),
        linewidth=options.get('linewidth', 2),
        fillstyle=options.get('fillstyle', 'full'),
        ms=options.get('size', 8))
    process_plot(fig, options)
    
def plot_fit(X, w, fig=None, options=dict()):
    if fig is None:
        fig = plt.subplot(111)

    x_min = np.min(X)
    x_max = np.max(X)
    dim = w.size - 1
    x_plot = np.reshape(np.linspace(x_min, x_max, 100), [-1, 1])
    x1_plot = np.ones_like(x_plot)
    for d in range(dim):
        x1_plot = np.concatenate((np.power(x_plot, 1 + d), x1_plot), axis=1)

    y_plot = np.dot(x1_plot, w)
    fig.plot(x_plot, y_plot, 'r-', label=options.get('label', 'Regression fit'))
    process_plot(fig, options)
    
def process_plot(fig, options=dict()):
    if 'x_label' in options.keys():
        fig.set_xlabel(options['x_label'])
    if 'y_label' in options.keys():
        fig.set_ylabel(options['y_label'])
    if 'x_lim' in options.keys():
        fig.set_ylim(options['x_lim'])
    if 'y_lim' in options.keys():
        fig.set_ylim(options['y_lim'])
    if 'title' in options.keys():
        fig.set_title(options['title'])
    if 'legend' in options.keys():
        if options['legend']:
            fig.legend(loc=options.get('legend_loc', 'best'))

## Generate the dataset


We generate our dataset according to the function model: $$x \sim \mathcal{N}(0, 1), \qquad y = \sin(2\pi x).$$

In [ ]:
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16

num_points = 10 # Number of training points.
noise = 0.5  # Noise Level (needed for data generation).

def generate_data(num_points, noise_level):
    # Function used for generating the true dataset
    x = np.linspace(0, 1, num_points).reshape(-1, 1)
    y = np.sin(2 * np.pi * x) + np.random.normal(scale=noise_level, size=(x.shape[0],1))
    return x, y

X, y = generate_data(num_points, noise)

# Plot Data
fig = plt.subplot(111);
plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Generated Data', 'y_lim': [np.min(y)-0.5, np.max(y)+0.5]}
plot_data(X, y, fig=fig, options=plot_opts)

## Generate polynomial feature matrix

In [ ]:
def generate_polynomial(x, degree):
    X = np.ones((x.shape[0], 1))
    for index in range(degree):
        X = np.concatenate([x**(index + 1), X], axis=1)
    return X

degree = 15
X_poly = generate_polynomial(X, degree=degree)

## Compute Closed Form Solution

In [ ]:
reg = 1e-3 # regularization lambda
dim = X_poly.shape[-1]
w_hat = np.dot(np.linalg.pinv(np.dot(X_poly.T, X_poly) + reg * np.eye(dim)), np.dot(X_poly.T, y))

In [ ]:
fig = plt.subplot(111)
plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Closed Form Solution', 'legend': False,
             'y_lim': [np.min(y)-1, np.max(y)+1]}

plot_data(X, y, fig=fig, options=plot_opts)
plot_fit(X, w_hat, fig=fig, options=plot_opts)